![image.png](../background_photos/libs_07_kargin.jpg)

[լուսանկարի հղումը](https://github.com/HaykTarkhanyan/python_math_ml_course/blob/main/background_photos/libs_07_kargin.jpg), Հեղինակ՝ չգիտեմ ով ա սարքել, բայց շատ շնորհակալ եմ իրան

<a href="ToDo" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> (ToDo)

> Song reference - ToDo

# 📌 Նկարագիր

[📚 Ամբողջական նյութը]()

#### 📺 Տեսանյութեր
#### 🏡 Տնային

# 📚 Նյութը

In [1]:
from pathlib import Path
import pandas as pd

XLSX_PATH = Path("assets", "kargin.xlsx")

# read first 2 sheets 
df1 = pd.read_excel(XLSX_PATH, sheet_name=0)
df2 = pd.read_excel(XLSX_PATH, sheet_name=1)

In [ ]:
!pip install fuzzywuzzy python-Levenshtein

In [2]:
cols = ['titles', 'links', 'Շատ տարածված արտահայտություն',
       'Տեքստ', 'Հիմնական դերասաններ', 'հիմնական դերասանների քանակ',
       'Դերերի անուններ', 'Վայր', 'Լուսավորվածություն', 'Լեզուներ', 'Արված է']

df1 = df1[cols]

In [3]:
df2.rename(columns={"Վայթ": "Վայր"}, inplace=True)

In [12]:
df2 = df2[cols]

In [18]:
res = pd.concat([df1, df2], ignore_index=True)
res = res[res["titles"].notna()]

res.to_csv("assets/kargin.csv", index=False)

# Eda

In [23]:
df = res 

In [25]:
df.columns

Index(['titles', 'links', 'Շատ տարածված արտահայտություն', 'Տեքստ',
       'Հիմնական դերասաններ', 'հիմնական դերասանների քանակ', 'Դերերի անուններ',
       'Վայր', 'Լուսավորվածություն', 'Լեզուներ', 'Արված է'],
      dtype='object')

In [27]:
df.rename(columns={
    "Շատ տարածված արտահայտություն": "text_common",
    "Տեքստ": "text",
    "Հիմնական դերասաններ": "main_actors",
    "հիմնական դերասանների քանակ": "main_actors_count",
    "Դերերի անուններ": "roles_names",
    "Վայր": "location",
    "Լուսավորվածություն": "lighting",
    "Լեզուներ": "languages",
    "Արված է": "done"
}, inplace=True)

In [29]:
res.to_csv("assets/kargin_eng.csv", index=False)

In [30]:
res.columns

Index(['titles', 'links', 'text_common', 'text', 'main_actors',
       'main_actors_count', 'roles_names', 'location', 'lighting', 'languages',
       'done'],
      dtype='object')

In [ ]:
import streamlit as st
from fuzzywuzzy import fuzz
import pandas as pd

def create_streamlit_app():
    st.title("Kargin Episode Search")
    
    # Load the data
    df = pd.read_csv("assets/kargin_eng.csv")
    
    # Text search
    search_text = st.text_input("Enter search text:")
    
    # Search type
    search_type = st.radio(
        "Search Type",
        ["Exact Match", "Fuzzy Search"],
        horizontal=True
    )
    
    # Fields to search in
    search_fields = st.multiselect(
        "Fields to search in",
        ["text", "text_common", "titles", "main_actors", "roles_names"],
        default=["text", "text_common"]
    )
    
    # Fuzzy match threshold
    if search_type == "Fuzzy Search":
        threshold = st.slider("Similarity Threshold", 50, 100, 70)
    
    # Filters
    st.sidebar.header("Filters")
    
    # Location filter
    locations = ['All'] + list(df['location'].dropna().unique())
    selected_location = st.sidebar.selectbox("Select Location", locations)
    
    # Language filter
    languages = ['All'] + list(df['languages'].dropna().unique())
    selected_language = st.sidebar.selectbox("Select Language", languages)
    
    # Main actors count filter
    min_actors = int(df['main_actors_count'].min())
    max_actors = int(df['main_actors_count'].max())
    selected_actors = st.sidebar.slider("Number of Main Actors", min_actors, max_actors, (min_actors, max_actors))
    
    # Apply filters
    filtered_df = df.copy()
    
    if selected_location != 'All':
        filtered_df = filtered_df[filtered_df['location'] == selected_location]
    
    if selected_language != 'All':
        filtered_df = filtered_df[filtered_df['languages'] == selected_language]
    
    filtered_df = filtered_df[
        (filtered_df['main_actors_count'] >= selected_actors[0]) & 
        (filtered_df['main_actors_count'] <= selected_actors[1])
    ]
    
    if search_text:
        results = []
        
        for _, row in filtered_df.iterrows():
            max_similarity = 0
            
            for field in search_fields:
                field_text = str(row[field]) if pd.notna(row[field]) else ""
                
                if search_type == "Exact Match":
                    if search_text.lower() in field_text.lower():
                        max_similarity = 100
                        break
                else:  # Fuzzy Search
                    similarity = fuzz.partial_ratio(search_text.lower(), field_text.lower())
                    max_similarity = max(max_similarity, similarity)
            
            if (search_type == "Exact Match" and max_similarity == 100) or \
               (search_type == "Fuzzy Search" and max_similarity >= threshold):
                results.append({
                    'row': row,
                    'similarity': max_similarity
                })
        
        # Sort results by similarity
        results.sort(key=lambda x: x['similarity'], reverse=True)
        
        # Display results
        if results:
            st.subheader(f"Search Results ({len(results)} found)")
            for result in results:
                row = result['row']
                st.markdown(f"### [{row['titles']}]({row['links']})")
                st.write(f"Match Score: {result['similarity']}%")
                st.write(f"Location: {row['location']}")
                st.write(f"Main Actors: {row['main_actors']}")
                if pd.notna(row['text_common']):
                    st.write(f"Common Expression: {row['text_common']}")
                st.write("---")
        else:
            st.warning("No matches found")
    else:
        st.info("Enter some text to search for episodes")

In [ ]:
if __name__ == "__main__":
    create_streamlit_app()

In [22]:
for c in res.columns:
    print(f"{c}: {res[c].unique()} unique values")

titles: ['Kargin Haghordum sketch 401 (Hayko Mko)'
 'Kargin Haghordum sketch 402 (Hayko Mko)'
 'Kargin Haghordum sketch 403 (Hayko Mko)'
 'Kargin Haghordum sketch 419 (Hayko Mko)'
 'Kargin Haghordum sketch 421 (Hayko Mko)'
 'Kargin Haghordum sketch 422 (Hayko Mko)'
 'Kargin Haghordum sketch 423 (Hayko Mko)'
 'Kargin Haghordum sketch 424 (Hayko Mko)'
 'Kargin Haghordum sketch 425 (Hayko Mko)'
 'Kargin Haghordum sketch 426 (Hayko Mko)'
 'Kargin Haghordum sketch 427 (Hayko Mko)'
 'Kargin Haghordum sketch 428 (Hayko Mko)'
 'Kargin Haghordum sketch 429 (Hayko Mko)'
 'Kargin Haghordum sketch 430 (Hayko Mko)'
 'Kargin Haghordum sketch 431 (Hayko Mko)'
 'Kargin Haghordum sketch 432 (Hayko Mko)'
 'Kargin Haghordum sketch 433 (Hayko Mko)'
 'Kargin Haghordum sketch 434 (Hayko Mko)'
 'Kargin Haghordum sketch 435 (Hayko Mko)'
 'Kargin Haghordum sketch 436 (Hayko Mko)'
 'Kargin Haghordum sketch 437 (Hayko Mko)'
 'Kargin Haghordum sketch 438 (Hayko Mko)'
 'Kargin Haghordum sketch 439 (Hayko Mko)'
 'K

# 🛠️ Գործնական


# 🏡Տնային


# 🎲 25 (07)
- ▶️[Cleo Abram](https://www.youtube.com/watch?v=bCmwCkNY85g)
- ▶️[Random link](https://www.youtube.com/watch?v=WnMv6-XTROY)
- 🇦🇲🎶[Մեր բակը](https://www.youtube.com/watch?v=ZrIML36hOLo)
- 🌐🎶[Կարգինում եղած երգերի playlist]()
- 🤌[Կարգին]()


<a href="http://s01.flagcounter.com/more/1oO"><img src="https://s01.flagcounter.com/count2/1oO/bg_FFFFFF/txt_000000/border_CCCCCC/columns_2/maxflags_10/viewers_0/labels_0/pageviews_1/flags_0/percent_0/" alt="Flag Counter"></a>
